In [1]:
# Prediction of Chronic Kidney Disease
import pandas as pd  
import numpy as np 
import matplotlib.pyplot as plt
dataset=pd.read_csv("CKD.csv") 
print("dataset available")
df = pd.DataFrame(dataset)
num_rows = len(df.index)
num_columns = len(df.columns)
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

dataset available
Number of rows: 399
Number of columns: 25


In [2]:
dataset.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hrmo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset=dataset.replace({True:1,False:0}) 

C:\Users\USER\AppData\Local\Temp\ipykernel_1112\4269844691.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset=dataset.replace({True:1,False:0})


In [5]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependent=dataset[['classification_yes']]

In [6]:
from sklearn.model_selection import train_test_split 
x_train3,x_test3,y_train3,y_test3 = train_test_split(independent,dependent, test_size=0.3,random_state=0) 

In [8]:
dataset["classification_yes"].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train3 = sc.fit_transform(x_train3)
x_test3 = sc.transform(x_test3)

In [11]:
x_train3.shape,x_test3.shape,y_train3.shape,y_test3.shape

((279, 27), (120, 27), (279, 1), (120, 1))

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier as RFC
param_grid = {'criterion':['entropy','gini'],'max_features':[1,'log2','sqrt'],'n_estimators':[10,100]}
grid = GridSearchCV(RFC(), param_grid, refit = True, verbose = 3,cv=5,n_jobs = -1 )
grid.fit(x_train3,y_train3)
print("Model Created")

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Model Created


C:\Anaconda\anaconda3\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [13]:
result = grid.cv_results_
grid_predictions =  grid.predict(x_test3)
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test3,grid_predictions) 
print("The cm vaule for best parameter {} is:'\n'".format(grid.best_params_),cm) 

The cm vaule for best parameter {'criterion': 'entropy', 'max_features': 1, 'n_estimators': 100} is:'
' [[45  0]
 [ 1 74]]


In [14]:
from sklearn.metrics import classification_report
cls_report = classification_report(y_test3,grid_predictions)
print ("The report for Random Forest Classification is: '\n'", cls_report)

The report for Random Forest Classification is: '
'               precision    recall  f1-score   support

           0       0.98      1.00      0.99        45
           1       1.00      0.99      0.99        75

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120



In [15]:
#result
table = pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.047810,0.009902,0.008179,0.000995,entropy,1,10,"{'criterion': 'entropy', 'max_features': 1, 'n...",1.000000,0.928571,0.964286,0.982143,1.000000,0.975000,0.026726,7
1,0.288370,0.013775,0.018532,0.000154,entropy,1,100,"{'criterion': 'entropy', 'max_features': 1, 'n...",1.000000,0.982143,0.982143,0.982143,1.000000,0.989286,0.008748,1
2,0.033689,0.002430,0.006273,0.000289,entropy,log2,10,"{'criterion': 'entropy', 'max_features': 'log2...",0.946429,0.982143,0.964286,0.982143,1.000000,0.975000,0.018211,7
3,0.283240,0.005314,0.018497,0.000536,entropy,log2,100,"{'criterion': 'entropy', 'max_features': 'log2...",1.000000,0.964286,0.946429,0.982143,1.000000,0.978571,0.020825,5
4,0.032774,0.001208,0.006350,0.000371,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.964286,0.982143,0.910714,0.964286,1.000000,0.964286,0.029881,11
5,0.290230,0.018224,0.017650,0.000202,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",1.000000,0.946429,0.928571,0.982143,1.000000,0.971429,0.029014,10
6,0.033013,0.005810,0.006178,0.000241,gini,1,10,"{'criterion': 'gini', 'max_features': 1, 'n_es...",1.000000,0.946429,0.982143,0.982143,0.981818,0.978506,0.017484,6
7,0.260049,0.002969,0.017945,0.000271,gini,1,100,"{'criterion': 'gini', 'max_features': 1, 'n_es...",0.982143,0.964286,0.982143,0.982143,1.000000,0.982143,0.011294,3
8,0.031312,0.000951,0.005942,0.000116,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",1.000000,0.964286,0.964286,0.964286,1.000000,0.978571,0.017496,4
9,0.262788,0.002068,0.016539,0.002406,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",1.000000,0.964286,0.982143,0.982143,1.000000,0.985714,0.013363,2


In [16]:
#F1 SCORE
from sklearn.metrics import f1_score
f1_macro =  f1_score(y_test3,grid_predictions,average='weighted')
print("The f1 score vaule for best parameter {} is:'\n'".format(grid.best_params_),f1_macro) 

The f1 score vaule for best parameter {'criterion': 'entropy', 'max_features': 1, 'n_estimators': 100} is:'
' 0.9916844900066377


In [17]:
#roc_auc Area Under Curve  1 is the best 
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test3,grid.predict_proba(x_test3)[:,1]) 

np.float64(0.9995555555555555)

In [19]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [25]:
age_input=float(input("age"))
bp_input=float(input("bp"))
al_input=float(input("al"))
su_input=float(input("su"))
bgr_input=float(input("bgr"))
bu_input=float(input("bu"))
sc_input=float(input("sc"))
sod_input=float(input("sod"))
pot_input=float(input("pot"))
hrmo_input=float(input("hrmo"))
pcv_input=float(input("pcv"))
wc_input=float(input("wc"))
rc_input=float(input("rc"))
sg_b_input=float(input("sg_b"))
sg_c_input=float(input("sg_c"))
sg_d_input=float(input("sg_d"))
sg_e_input=float(input("sg_e"))
rbc_input=float(input("rbc_normal"))
pc_input=float(input("pc_normal"))
pcc_input=float(input("pcc_present"))
ba_input=float(input("ba_present"))
htn_input=float(input("htn_yes"))
dm_input=float(input("dm_yes"))
cad_input=float(input("cad_yes"))
appet_input=float(input("appet_yes"))
pe_input=float(input("pe_yes"))
ane_input=float(input("ane_yes"))

age 12
bp 80
al 0
su 0
bgr 100
bu 26
sc 0.6
sod 137
pot 4.4
hrmo 15.8
pcv 49
wc 6600
rc 5.4
sg_b 0
sg_c 0
sg_d 0
sg_e 0
rbc_normal 1
pc_normal 1
pcc_present 0
ba_present 0
htn_yes 0
dm_yes 0
cad_yes 0
appet_yes 1
pe_yes 1
ane_yes 0


In [26]:
Future_Prediction=grid.predict([[age_input,bp_input,al_input,su_input,bgr_input,bu_input,sc_input,sod_input,pot_input,hrmo_input,pcv_input,wc_input,
                                 rc_input,sg_b_input,sg_c_input,sg_d_input,sg_e_input,rbc_input,pc_input,pcc_input,ba_input,htn_input,dm_input,
                                 cad_input,appet_input,pe_input,ane_input]])
print("Future_Prediction={}".format(Future_Prediction))
print("The possibility of Kidney Disease is:",Future_Prediction)

Future_Prediction=[1]
The possibility of Kidney Disease is: [1]


In [20]:
grid

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_features': [1, 'log2', 'sqrt'],
                         'n_estimators': [10, 100]},
             verbose=3)